In [ ]:
# Author: Bryce Cyr

# This notebook allows one to study the effects of soft photon heating as described in 2024.xxxx.
# To use: modify relevant sections in the 'User Input Section' cell, and then run rest of the cells.
# Plots of T_M and X_e compare against pre-tabulated solutions from CosmoRec (vanilla LCDM).
# The cosmos.py contains details of most of the function calls used by the solver.
# See the readme at https://github.com/Bryce-Cyr/Soft_Photon_Heating for more info.

In [ ]:
%matplotlib notebook

import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import cosmos as c

from scipy.integrate import solve_ivp
from scipy import integrate
from scipy.special import gamma

In [ ]:
# -------------------------------------------------- # 
#                User Input Section                  #
# -------------------------------------------------- # 

###### Switches #######
inc_CS   = 1;              # Include Compton interactions
inc_Src  = 1;              # Include a source term
##### Select one ######
gen_Src  = 1;              # Use generic source term (Eq. 21)
RSB_Src  = 0;              # Use RSB source term (Eq. 27)
##### Select one ######
set_A    = 0;              # Set the amplitude of source (internally compute drho/rho)
set_drho = 1;              # Set drho/rho of source (internally compute the amplitude)
#######################

                
# Select (up to 6) values of redshift where you would like to save the spectrum
z_view = [490,495,498]; 


#--------------------------------------------------------------------------#
#----------- Generic source term model (from 2024.xxxx, Eq. 21) -----------#
#--------------------------------------------------------------------------#
Gen_zinj  = 500;        # [0]   Redshift of injection (integer 1 < z < 1500)
Gen_Ecut  = 0.235e-9;   # [GeV] High energy cutoff (< 10 eV)
Gen_gamma = 3.6;        # [0]   Spectral index (3.0 = free-free, 3.6 = synchrotron)

if set_A == 1:
    Gen_A = 1.85e-7;    # [0]   Dimensionless amplitude of spectrum
    
    # Derived (don't change)
    Gen_drho = (Gen_A*15*gamma(4-Gen_gamma))/(np.pi**4)* \
               (Gen_Ecut/c.T_CMB(Gen_zinj))**4

if set_drho == 1:
    Gen_drho = 1e-6;    # [0]   Fractional energy injection drho/rho
    
    # Derived (don't change)
    Gen_A    = (np.pi**4)/(15*gamma(4-Gen_gamma))* \
               ((c.T_CMB(Gen_zinj)/Gen_Ecut)**4)*Gen_drho
#--------------------------------------------------------------------------#
#--------------------------------------------------------------------------#
#--------------------------------------------------------------------------#



#--------------------------------------------------------------------------#
#------------ RSB source term model (from 2024.xxxx, Eq. 27) --------------#
#--------------------------------------------------------------------------#
RSB_zinj  = 500;        # [0]   Redshift of injection (integer 1 < z < 1500)
RSB_Ecut  = 0.235e-9;   # [GeV] High energy cutoff (RJ limit: Ecut/kb T(zinj) < 1)
RSB_E0    = 0.235e-10;  # [GeV] Pivot energy (= h \nu_0) at redshift = 0 (<= Ecut)
RSB_gamma = 3.6;        # [0]   Spectral index (3.0 = free-free, 3.6 = synchrotron)

if set_A == 1:
    RSB_Amp    = 0.45;  # [0]   Ratio of T_amp/T_CMB (redshift independent)
    
    # Derived (don't change)
    RSB_Atilde = RSB_Amp*(c.T_CMB(0)/RSB_E0)*(RSB_Ecut/RSB_E0)**(-RSB_gamma)
    RSB_drho   = (RSB_Atilde*15*gamma(4-RSB_gamma))/(np.pi**4)* \
                 (RSB_Ecut/c.T_CMB(RSB_zinj))**4

if set_drho == 1:
    RSB_drho = 1e-6;    # [0]   Fractional energy injection drho/rho
    
    # Derived (don't change)
    RSB_Atilde = (np.pi**4)/(15*gamma(4-Gen_gamma))* \
                 ((c.T_CMB(Gen_zinj)/Gen_Ecut)**4)*Gen_drho    
    RSB_Amp = RSB_Atilde/((c.T_CMB(0)/RSB_E0)*(RSB_Ecut/RSB_E0)**(-RSB_gamma))
#--------------------------------------------------------------------------#
#--------------------------------------------------------------------------#
#--------------------------------------------------------------------------#



#--------------------------------------------------------------------------#
#------ These are things that the user doesn't usually need to modify -----#
#--------------------------------------------------------------------------#
# Defining frequency grid (x = h\nu/kT(z))
x_min = 1e-8;           # Default 1e-8
x_max = 1e2;            # Default 1e2
x_fidel = 5000;         # Default = 5000 (minimum 2000 for FF integrals)
x = np.logspace(np.log10(x_min),np.log10(x_max),x_fidel);


# Redshift range of the solution
z_start = 1500          # Starting redshift, default = 1500
z_end   = 1             # Ending redshift, default = 1 (warning, no reionization module!)
z_step  = 0.2           # Only works for 0.1, 0.2, 0.5, 1.0. Don't go z > 1. Larger drho require smaller z_step
z = np.linspace(z_end,z_start,int((z_start-z_end)/z_step)+1) 

if gen_Src == 1:
    Src_zinj  = Gen_zinj
    Src_Ecut  = Gen_Ecut
    Src_gamma = Gen_gamma
    Src_A     = Gen_A
    Src_drho  = Gen_drho
    
if RSB_Src == 1:
    Src_zinj  = RSB_zinj
    Src_Ecut  = RSB_Ecut
    Src_gamma = RSB_gamma
    Src_A     = RSB_Atilde
    Src_drho  = RSB_drho
#--------------------------------------------------------------------------#
#--------------------------------------------------------------------------#
#--------------------------------------------------------------------------#



#--------------------------------------------------------------------------#
#------------- Throwing various warnings to the user ----------------------#
#--------------------------------------------------------------------------#
if RSB_Ecut/c.T_CMB(RSB_zinj) >= 1 and RSB_Src == 1:
    print("Warning! RSB injection not in RJ limit (x < 1)! x_cut = "+f"{round(RSB_Ecut/c.T_CMB(RSB_zinj),3)}")
    
if RSB_Ecut/RSB_E0 < 1 and RSB_Src == 1:
    print("Warning! Pivot frequency greater than cutoff frequency!")
    
if RSB_drho > 1e-4 and RSB_Src == 1:
    RSB_drho_form="{:.3e}".format(RSB_drho)
    print("Warning! Large energy injection! drho = " + f"{round(RSB_drho_form,3)}")
    
if RSB_drho > 1e-4 and gen_Src == 1:
    Gen_drho_form="{:.3e}".format(Gen_drho)
    print("Warning! Large energy injection! drho = " + f"{round(Gen_drho_form,3)}")
    
if gen_Src == 1 and RSB_Src == 1:
    print("Warning! Both generic source and RSB selected!")
    
if gen_Src == 0 and RSB_Src == 0 and inc_Src == 1:
    print("Warning! Please specify a form for the source term!")
    
if set_A == 1 and set_drho == 1:
    print("Warning! Please set either A OR drho!")
    
if gen_Src == 0 and RSB_Src == 0 and inc_Src == 1:
    print("Warning! Please set either A or drho!")
#--------------------------------------------------------------------------#
#--------------------------------------------------------------------------#
#--------------------------------------------------------------------------#

In [ ]:
#--------------------------------------------------------------#
#                                                              #
#              Run the following cells in order                #
#                                                              #
#--------------------------------------------------------------#

In [ ]:
# -------------------------------------------------- # 
#                 Main solver                        #
# -------------------------------------------------- #

# Y[0] = X_e
# Y0[0] = X_e[z = z_max] = 1
# Y1 = T_M
# Y0[1] = T_M[z = z_max] = T_CMB(z_max) (approx)
# The spectral distortion is computed analytically at each timestep

# ---------- Defining initial conditions for diff eq ---------- #
# Distortion ICs
init_Dist = np.zeros(len(x))
Dist = np.zeros(len(x))
Adv_Dist = np.zeros(len(x))

# X_e and T_M ICs
# T_M_ini is the approx electron temp valid from 500 < z < 1500
T_M_ini = c.T_CMB(z_start)*(1+((1+1+c.f_HE)*3*c.m_e*c.H_z(z_start))/(1*8*c.sigma_T*c.rho_CMB(z_start)))**(-1)
Y1 = [1]
Y2 = [T_M_ini]


# ---------- Initializing other quantities of interest ---------- #
# Initializing when to save intermediate distortions
Int_red = np.zeros((len(z_view),len(x)))

# Soft photon heating term
SPH_int = [0]

# To extract additional heating terms at each step
Hubble_int  = [0]
Compton_int = [0]

# Initializing injection term 
S_inj = np.zeros(len(x))
# ------------------------------------------------------------- #

# Outer loop goes from i = high -> low in steps of -1
for i in range(z_start,z_end,-1):
    
    # Inner loop sets the number and size of timesteps over which to inject source term
    for j in range(0,int(1/z_step),1):
        loop_z_start = i - j*z_step
        loop_z_end   = i - j*z_step - z_step

    
        Y0 = [Y1[-1],Y2[-1]]
        answer = solve_ivp(c.rhs, [loop_z_start,loop_z_end], Y0, method='BDF', dense_output=True, args = (SPH_int[-1],))
        z_sol = np.linspace(loop_z_end,loop_z_start,10)
        Y_stepper = answer.sol(z_sol)
        Y1.append(Y_stepper[0,0])
        Y2.append(Y_stepper[1,0])
        
        #------------------- QI source term injection -------------------#
        # The injection is smoothed over Delta_z = 1 (inner loop size).
        # If solver not converging, increase z_step
        if inc_Src == 1 and np.abs(i-Src_zinj) == 0:
            Src_xcut = Src_Ecut/c.T_CMB(i)
            Src_term  = Src_A*((x/Src_xcut)**(-Src_gamma))*np.exp(-x/Src_xcut)
            Src = Src_term*z_step
        else:
            Src_term = np.zeros(len(x))
            Src = Src_term
        #------------------------------------------------------------#
        

        #---------- Determine n(x) evolution  (vectorized) ----------#    
        S_inj = Src
        Adv_Dist[:] = Dist[:]*np.exp(-c.Delta_tau_ff(x[:], loop_z_start, loop_z_end, Y2[-1], Y1[-1]))+ \
                      c.Delta_Src_tilde(x[:], loop_z_start, loop_z_end, Y2[-1], Y1[-1], S_inj[:], inc_CS, inc_Src)* \
                      (1-np.exp(-c.Delta_tau_ff(x[:], loop_z_start, loop_z_end, Y2[-1], Y1[-1])))

        SPH_int_term = c.Compute_heating(Adv_Dist, x, loop_z_start, loop_z_end, Y2[-1], Y1[-1])
        SPH_int.append(SPH_int_term)
        
        Dist = Adv_Dist
        #------------------------------------------------------------#
        
        
        # Store distortion at intermediate redshifts selected above  #
        for k in range(len(z_view)):
            if z_view[k] == loop_z_end:
                Int_red[k] = Dist
        #------------------------------------------------------------#
        
        
        #---------- Save Hubble and Compton heating terms -----------#
        Hubble_int.append(2*Y2[-1]/(1+loop_z_end))
        Compton_int.append((Y1[-1])/(1+Y1[-1]+c.f_HE)*(8*c.sigma_T)/(3*c.m_e)* \
                          c.rho_CMB(loop_z_end)/((1+loop_z_end)*c.H_z(loop_z_end))* \
                          (Y2[-1]-c.T_CMB(loop_z_end)))
        #------------------------------------------------------------#

        

# Swapping order of solutions so that values go from low to high redshift 
SPH_int = SPH_int[::-1]
Hubble_int = Hubble_int[::-1]
Compton_int = Compton_int[::-1]
Y1 = Y1[::-1]
Y2 = Y2[::-1]

# Now, we have
# Y1[0] = X_e[z_min]
# Y2[0] = T_m[z_min]


In [ ]:
# -------------------------------------------------- # 
#        Heating rates (dT_ff/dz in kelvin)          #
# -------------------------------------------------- # 

###### Select one ######
z_plot   = 0;
lnz_plot = 1;
########################

plt.close('all')
fig, ax = plt.subplots()

# Convert instantaneous heating terms from GeV to K
SPH_int_K = np.divide(SPH_int,c.kb)
Hubble_int_K = np.divide(Hubble_int,c.kb)
Compton_int_K = np.divide(Compton_int,c.kb)

if z_plot == 1:   
    plt.loglog(z, Hubble_int_K,'-', color = 'red', label = "Hubble") 
    plt.loglog(z, -Hubble_int_K,'--', color = 'red')
    
    plt.loglog(z, Compton_int_K,'-', color = 'blue', label = "Compton") 
    plt.loglog(z, -Compton_int_K,'--', color = 'blue')
    
    plt.loglog(z, SPH_int_K,'-', color = 'green', label = "Bremsstrahlung") 
    plt.loglog(z, -SPH_int_K,'--', color = 'green')
    
    plt.ylabel('${\\rm d}T_{\\rm M,i}/{\\rm d}z$ (K)')
    
if lnz_plot == 1:    
    plt.loglog(z, Hubble_int_K*z,'-', color = 'red', label = "Hubble") 
    plt.loglog(z, -Hubble_int_K*z,'--', color = 'red')
    
    plt.loglog(z, Compton_int_K*z,'-', color = 'blue', label = "Compton") 
    plt.loglog(z, -Compton_int_K*z,'--', color = 'blue')
    
    plt.loglog(z, SPH_int_K*z,'-', color = 'green', label = "Bremsstrahlung") 
    plt.loglog(z, -SPH_int_K*z,'--', color = 'green')
    
    plt.ylabel('${\\rm d}T_{\\rm M,i}/{\\rm dln}(z)$ (K)')

# ------------------- Generic labels -------------------------- #
plt.loglog(z,1e-50*z/z,'-', color = 'black', label = "Matter Cooling")
plt.loglog(z,1e-50*z/z,'-', color = 'black', label = "Matter Heating", linestyle = "--")
# ------------------------------------------------------------- #

# ------------------- Plot annotations ------------------------ #
Src_drho_form="{:.2e}".format(Src_drho)

plt.text(1.4e1,1e8,f"Single injection: $\\gamma = {Src_gamma}$", size = 11)
plt.text(8.5e1,1e7,"$\\frac{\\Delta \\rho}{\\rho}$" + f"= {Src_drho_form}", size = 11)
plt.text(7.8e1,1e6,"$z_{\\rm inj}$" + f"= {Src_zinj}", size = 11)

plt.grid(linestyle = '--', alpha = 0.2, which = "minor")
plt.grid(linestyle = '-', alpha = 0.2, which = "major")
ax.yaxis.grid(False, which = 'minor')

plt.legend(loc='upper left')
plt.xlabel('$z$')


plt.xlim([z_end,z_start])
plt.ylim([1e-5,1e9])
# ------------------------------------------------------------- #

In [ ]:
# -------------------------------------------------- # 
#           Matter temperature evolution             #
# -------------------------------------------------- #


# ----------- Loading in CosmoRec (no inj) solutions ---------- #
CosmoRecDataName = "CosmoRec.Sol.dat"

CRD = pd.read_csv(CosmoRecDataName, 
        sep=" |\t", skiprows = 2, engine="python", 
        names = ["z","Xe=Ne/NH","XHI","XHeI","XHeII","Te"])

CRDFrame = pd.DataFrame(CRD);
# ------------------------------------------------------------- #

plt.close('all')
fig, ax = plt.subplots()

# Convert Matter temperature from GeV to K
Y2K = np.divide(Y2,c.kb)

# Result from the solver (including SPH)
plt.loglog(z, Y2K,'-', label='$T_{\\rm M}(z)$', color = 'orange') 

# Plot the CMB temperature for comparison
plt.loglog(z,c.T_CMB(z)/c.kb, label = '$T_{\\rm CMB}$')

# CosmoRec (no injection) solution for T_M (only goes to z = 100)
xValRec = CRDFrame["z"].to_numpy()
yValRec = CRDFrame["Te"].to_numpy()

plt.loglog(xValRec,yValRec, color = 'black', linestyle = '--', alpha = 1, label = "CosmoRec")


# ------------------- Plot annotations ------------------------ #
Src_drho_form="{:.2e}".format(Src_drho)

plt.text(1.4e0,5e4,f"Single injection: $\\gamma = {Src_gamma}$", size = 11)
plt.text(8.5e0,2.7e4,"$\\frac{\\Delta \\rho}{\\rho}$" + f"= {Src_drho_form}", size = 11)
plt.text(7.8e0,1.6e4,"$z_{\\rm inj}$" + f"= {Src_zinj}", size = 11)


plt.grid(linestyle = '--', alpha = 0.2, which = "minor")
plt.grid(linestyle = '-', alpha = 0.2, which = "major")
ax.yaxis.grid(False, which = 'minor')

plt.legend(loc='upper right')
plt.xlabel('$z$')
plt.ylabel('$T_{\\rm M}$ (K)')

plt.xlim([z_end,z_start])
plt.ylim([1e1,1e5])
# ------------------------------------------------------------- #

In [ ]:
# -------------------------------------------------- # 
#          Ionization fraction evolution             #
# -------------------------------------------------- #

plt.close('all')
fig, ax = plt.subplots()

# ----------- Loading in CosmoRec (no inj) solutions ---------- #
CosmoRecDataName = "CosmoRec.Sol.dat"

CRD = pd.read_csv(CosmoRecDataName, 
        sep=" |\t", skiprows = 2, engine="python", 
        names = ["z","Xe=Ne/NH","XHI","XHeI","XHeII","Te"])

CRDFrame = pd.DataFrame(CRD);
# ------------------------------------------------------------- #

# Result from the solver (including SPH)
plt.loglog(z, Y1,'-', label='$X_{\\rm e}(z)$', color = 'orange') 

# CosmoRec (no injection) solution for X_e (only goes to z = 100)
xValRec = CRDFrame["z"].to_numpy()
yValRec = CRDFrame["Xe=Ne/NH"].to_numpy()

plt.loglog(xValRec,yValRec, color = 'black', linestyle = '--', alpha = 1, label = "CosmoRec")

# ------------------- Plot annotations ------------------------ #
Src_drho_form="{:.2e}".format(Src_drho)

plt.text(1.4e1,1e-0,f"Single injection: $\\gamma = {Src_gamma}$", size = 11)
plt.text(8.5e1,5e-1,"$\\frac{\\Delta \\rho}{\\rho}$" + f"= {Src_drho_form}", size = 11)
plt.text(7.8e1,2.6e-1,"$z_{\\rm inj}$" + f"= {Src_zinj}", size = 11)


plt.grid(linestyle = '--', alpha = 0.2, which = "minor")
plt.grid(linestyle = '-', alpha = 0.2, which = "major")
ax.yaxis.grid(False, which = 'minor')

plt.legend(loc='upper left')
plt.xlabel('$z$')
plt.ylabel('$X_{\\rm e}$')

plt.xlim([z_end,z_start])
plt.ylim([1e-4,2])
# ------------------------------------------------------------- #

In [ ]:
# -------------------------------------------------- # 
#                Distortion spectra                  #
# -------------------------------------------------- #

###### Switches #######
plot_CMB      = 1;        # Plot the CMB for comparison
plot_Src      = 1;        # Plot the initial injected source 
plot_Int_Dist = 1;        # Plot the intermediate distortions
plot_Fin_Dist = 1;        # Plot the distortion at z_end
#######################

plt.close('all')
fig, ax = plt.subplots()

# Plots different colours for the intermediate redshifts
colWheel = ['orange', 'green', 'purple', 'grey', 'teal', 'brown']


# --------------- Plotting the distortions -------------------- #
# Distortion at z_end
spec_final = Dist

# Positive branches
if plot_Fin_Dist == 1:
    plt.loglog(x, spec_final,'-', color = 'blue', label = f"z = {np.round(z[0])} ") 
if plot_Int_Dist == 1:
    for i in range(len(z_view)):
        plt.loglog(x, Int_red[i],'-', color = colWheel[i], label = f"z = {z_view[i]} ")

# Negative branches
if plot_Fin_Dist == 1:
    plt.loglog(x, -spec_final,'--', color = 'blue') 
if plot_Int_Dist == 1:
    for i in range(len(z_view)):
        plt.loglog(x, -Int_red[i],'--', color = colWheel[i])

# Plotting n(x) for CMB to compare
if plot_CMB == 1:
    plt.loglog(x,c.n_BB(x),'-', color = 'black', label = "CMB")

# Plot the initially injected source term
Src_xcut = Src_Ecut/c.T_CMB(Src_zinj)
Src_val  = Src_A*((x/Src_xcut)**(-Src_gamma))*np.exp(-x/Src_xcut)
if plot_Src == 1:
    plt.loglog(x,Src_val,'-', color = 'magenta', label = f"Initial Inj at z = {Src_zinj}")
# ------------------------------------------------------------- #

# ------------------- Plot annotations ------------------------ #
Src_drho_form="{:.2e}".format(Src_drho)

plt.text(1e-6,1e-12,f"Single injection: $\\gamma = {Src_gamma}$", size = 11)
plt.text(3e-4,5e-16,"$\\frac{\\Delta \\rho}{\\rho}$" + f"= {Src_drho_form}", size = 11)
plt.text(2.4e-4,2.6e-19,"$z_{\\rm inj}$" + f"= {Src_zinj}", size = 11)


plt.grid(linestyle = '--', alpha = 0.2, which = "minor")
plt.grid(linestyle = '-', alpha = 0.2, which = "major")
ax.yaxis.grid(False, which = 'minor')

plt.legend(loc='upper right')
plt.xlabel('$x$')
plt.ylabel('$\\Delta n(x)$')

plt.xlim([x_min,x_max])
plt.ylim([1e-25,1e25])
# ------------------------------------------------------------- #

In [ ]:
# -------------------------------------------------- # 
#  Dim-less total intensity spectra I/I0 = x^3 n(x)  #
# -------------------------------------------------- #

###### Switches #######
plot_SRC       = 1;        # Plot the initial injected spectrum 
plot_Int_Dist  = 1;        # Plot the intermediate distortions
plot_Fin_Dist  = 1;        # Plot the distortion at z_end
#######################

##### Select one ######
plot_total_int =  1;       # Plot I/I_0 = x^3 n(x)
plot_dist_int  =  0;       # Plot I/I_0 = x^3 Delta n(x)
#######################

# Plots different colours for the intermediate redshifts
colWheel = ['orange', 'green', 'purple', 'grey', 'teal', 'brown']

plt.close('all')
fig, ax = plt.subplots()

# --------------- Plotting the intensities -------------------- #
# Blackbody intensity spectrum (redshift independent)
I_BB_final = (x**3)*c.n_BB(x)

# Computing blackbody at the electron temperature
x_e_final = x*c.T_CMB(z_end)/Y2[0]
I_BB_final_elec = (x**3)*c.n_BB(x_e_final)

# Distortion intensity spectrum
I_dist_final = (x**3)*Dist
I_dist_init = (x**3)*init_Dist

# Total intensity spectrum
I_tot_final = I_BB_final + I_dist_final
I_tot_init  = I_BB_final + I_dist_init

# Note: no negative branches for total intensity
if plot_total_int == 1:
    if plot_Fin_Dist == 1:
        plt.loglog(x,I_tot_final,'-', color = 'blue', label = f"z = {np.round(z[0])}")
    if plot_Int_Dist == 1:
        for i in range(len(z_view)):
            plt.loglog(x, I_BB_final+(x**3)*Int_red[i],'-', color = colWheel[i], label = f"z = {z_view[i]} ")
            
    plt.ylabel('$x^3 (n_{\\rm bb} + \\Delta n)$')
    
if plot_dist_int == 1:  
    if plot_Fin_Dist == 1:
        plt.loglog(x,I_dist_final,'-', color = 'blue', label = f"z = {np.round(z[0])}")
    if plot_Int_Dist == 1:
        for i in range(len(z_view)):
            plt.loglog(x,(x**3)*Int_red[i],'-', color = colWheel[i], label = f"z = {z_view[i]} ")   
    
    # negative branches 
    if plot_Fin_Dist == 1:
        plt.loglog(x,-I_dist_final,'--', color = 'blue')
    if plot_Int_Dist == 1:
        for i in range(len(z_view)):
            plt.loglog(x,-Int_red[i],'--', color = colWheel[i])                    
    
    plt.ylabel('$x^3 \\Delta n$')

    
# I/I0 BB spectrum (valid all z)
plt.loglog(x,I_BB_final,'-', color = 'black', label = "Blackbody at $T_{\\rm CMB}$")
# I/I0 BB spectrum at electron temp
plt.loglog(x,I_BB_final_elec,'-', color = 'teal', label = "Blackbody at $T = T_{\\rm M}$"+f"(z={z[0]})")


# Plot initially injected source term
Src_xcut = Src_Ecut/c.T_CMB(Src_zinj)
Src_val  = Src_A*((x/Src_xcut)**(-Src_gamma))*np.exp(-x/Src_xcut)
if plot_Src == 1:
    plt.loglog(x,(x**3)*Src_val+I_BB_final,'-', color = 'magenta', label = f"Initial Inj at z = {Src_zinj}")
# ------------------------------------------------------------- #
    
# ------------------- Plot annotations ------------------------ #
Src_drho_form="{:.2e}".format(Src_drho)

plt.text(5.7e-7,2e5,f"Single injection: $\\gamma = {Src_gamma}$", size = 11)
plt.text(3e-4,2e3,"$\\frac{\\Delta \\rho}{\\rho}$" + f"= {Src_drho_form}", size = 11)
plt.text(2.4e-4,2.6e1,"$z_{\\rm inj}$" + f"= {Src_zinj}", size = 11)


plt.grid(linestyle = '--', alpha = 0.2, which = "minor")
plt.grid(linestyle = '-', alpha = 0.2, which = "major")
ax.yaxis.grid(False, which = 'minor')

plt.legend(loc='lower center')
plt.xlabel('$x$')

plt.xlim([x_min,x_max])
plt.ylim([1e-25,1e7])
# ------------------------------------------------------------- #